In [279]:
import os
import numpy as np
import pandas as pd
import glob
from pathlib import Path
from skimage.io import ImageCollection, imread
from skimage import img_as_ubyte
from tqdm import tqdm
import trackpy as tp
import matplotlib.pyplot as plt
from pathlib import Path
import cv2
%matplotlib qt

In [280]:
data_path = Path("/Users/jack/Documents/one-cell")

nuclei_path = data_path / "nuclei.tif"
cyto_path = data_path / "cyto.tif"
pattern_path = data_path / "patterns.tif"

nuclei = ImageCollection(str(nuclei_path))
cyto = ImageCollection(str(cyto_path))
pattern = img_as_ubyte(imread(str(pattern_path)))

In [281]:
def area(bbox):
    x1, y1, w, h = bbox
    return w * h

In [282]:
def adjust(bbox, height=0):
    x1, y1, w, h = bbox
    new_bbox = (x1, y1 + (h - height)//2, w, height)
    return new_bbox

In [283]:
def crop(image, bbox):
    x1, y1, w, h = bbox
    return image[y1:y1+h, x1:x1+w]

In [284]:
pattern_blur = cv2.GaussianBlur(pattern, (5, 5), 0)
_, pattern_binary = cv2.threshold(pattern_blur, 1, 255, cv2.THRESH_BINARY)
pattern_binary = cv2.morphologyEx(pattern_binary, cv2.MORPH_ERODE, np.ones((5, 5), np.uint8))
pattern_rgb = cv2.cvtColor(pattern_binary, cv2.COLOR_GRAY2RGB)
contours, _ = cv2.findContours(pattern_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
bbox = [cv2.boundingRect(c) for c in contours]
bbox = [b for b in bbox if area(b) > 1000]
bbox = [adjust(b, 30) for b in bbox]
pattern_rgb = cv2.cvtColor(pattern_binary, cv2.COLOR_GRAY2RGB)
# for b in bbox:
#     x1, y1, w, h = b
#     cv2.rectangle(pattern_rgb, (x1, y1), (x1 + w, y1 + h), (0, 255, 0), 2)
# plt.imshow(pattern_rgb)
# plt.show()

In [285]:
def normalize(image):
    image = img_as_ubyte(image)
    image_bg = cv2.medianBlur(image, 51)
    image_bg_blur = cv2.GaussianBlur(image_bg, (0, 0), sigmaX=101, sigmaY=101)
    image_clean = cv2.subtract(image, image_bg_blur)
    p1 = np.percentile(image_clean[image_clean > 0], 1)
    p99 = np.percentile(image_clean[image_clean > 0], 99)
    image = (image_clean - p1) / (p99 - p1)
    image = image.clip(0, 1)
    image = image * 255
    return image.astype(np.uint8)

In [290]:
def normalize_simple(image):
    image = img_as_ubyte(image)
    min = np.min(image)
    max = np.max(image)
    image = (image - min) / (max - min)
    image = image * 255
    return image.astype(np.uint8)

In [286]:
# cyto_norm = normalize(cyto[0])
cyto_img = img_as_ubyte(cyto[0])
bg = cv2.medianBlur(cyto_img, 51)
bg_blur = cv2.GaussianBlur(bg, (0, 0), sigmaX=101, sigmaY=101)
cyto_clean = cv2.subtract(cyto_img, bg_blur)
fig, ax = plt.subplots(1, 2)
ax[0].imshow(cyto_img)
ax[1].imshow(cyto_clean)
plt.show()

/Users/jack/miniconda3/lib/python3.12/site-packages/skimage/io/collection.py:316: FutureWarning: The plugin infrastructure in `skimage.io` is deprecated since version 0.25 and will be removed in 0.27 (or later). To avoid this warning, please do not pass additional keyword arguments for plugins (`**plugin_args`). Instead, use `imageio` or other I/O packages directly. See also `skimage.io.imread`.
  self.data[idx] = self.load_func(fname, **kwargs)


In [291]:
overlay_rgb = np.zeros((*pattern_rgb.shape[:2], 3), dtype=np.uint8)
overlay_rgb[..., 0] = normalize(nuclei[0])
overlay_rgb[..., 1] = normalize_simple(cyto[0])
plt.imshow(crop(overlay_rgb, bbox[0]))
plt.show()

Testing to choose the best parameters

In [288]:
# Change the frame number, the diameter, and the separation value to check that the nuclei are recognized.
# Set topn to the number of cells to detect.

frames = imread(os.path.abspath(files[1]))
print(len(frames))
frame_no = 20
# f = tp.locate(frames[frame_no], diameter=25, minmass=0, topn=3, separation=7)
f = tp.locate(frames[frame_no], diameter=25, topn=3, separation=7)
print(f)
# tp.annotate(f, frames[frame_no])
# # plt.hist(f['mass'])
# plt.show()


NameError: name 'files' is not defined

Running trackpy on all files

In [47]:
# Parameters

## Tracking parameters
diameter = 25  #31 is better for 1 cell
separation = 7
topn = 3 # Number of cell
verbose = True

## Saving parameters
start_position = 1
stop_position = 1
overwrite = False
old_suffix = '-Hoechst.tif'
new_suffix = '-tracking.csv'

In [48]:
files = glob.glob(os.path.join(data_path, "*-Hoechst.tif"))

for filename in tqdm(files):
    print(filename)
    print(os.path.basename(filename)[:3])
    position = int(os.path.basename(filename)[:3])
    if position>=start_position and position<=stop_position:

        frames = imread(os.path.abspath(filename))

        t = track(frames, diameter=diameter, separation=separation, 
                           topn=topn, verbose=verbose)

        savename = os.path.basename(filename).removesuffix(old_suffix) + new_suffix
        savename = os.path.join(save_path, savename)
        if os.path.exists(savename):
            if overwrite:
                print(f"Warning: the file {savename} already exists and will be deleted.")
                os.remove(savename)
                t.to_csv(savename)
            else:
                print(f"Warning: the file {savename} already exists, the data will not be saved.")
        else:
            t.to_csv(savename)

100%|██████████| 2/2 [00:01<00:00,  1.26it/s]

Frame 130: 3 trajectories present.
Tracking of nuclei completed.
